In [8]:
import numpy as np
import pandas as pd
import glob
from os import listdir

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [11]:
def create_dataframe(folder_name):
    
    dataset = pd.DataFrame(columns=['image', 'label'])
    folders = glob.glob('./dataset/{}/images/*'.format(folder_name))
    for folder in folders:
        classes = glob.glob('{}/*.jpg'.format(folder))
        for image in classes:
            label = str(folder).split('\\')[1]
            photo = load_img(image, target_size=(32, 32))
            photo = img_to_array(photo)
            dataset = dataset.append({'image': photo, 'label': label}, ignore_index=True)
    return dataset

In [12]:
create_dataframe('train')

,image,label
0,"[[[181.0, 187.0, 199.0], [188.0, 196.0, 207.0]...",aeroplane
1,"[[[91.0, 108.0, 134.0], [90.0, 110.0, 135.0], ...",aeroplane
2,"[[[254.0, 254.0, 254.0], [255.0, 255.0, 255.0]...",aeroplane
3,"[[[81.0, 150.0, 227.0], [83.0, 149.0, 227.0], ...",aeroplane
4,"[[[72.0, 94.0, 151.0], [74.0, 99.0, 155.0], [7...",aeroplane
...,...,...
527,"[[[92.0, 107.0, 104.0], [104.0, 114.0, 113.0],...",tvmonitor
528,"[[[154.0, 152.0, 140.0], [161.0, 159.0, 146.0]...",tvmonitor
529,"[[[204.0, 189.0, 160.0], [210.0, 196.0, 169.0]...",tvmonitor
530,"[[[64.0, 42.0, 31.0], [115.0, 67.0, 31.0], [15...",tvmonitor
